# %% [markdown]

 # Mosaico de Fotos

 En este notebook, aprenderemos cómo unir dos imágenes para crear un mosaico o panorama. Utilizaremos técnicas de detección de características, coincidencia de puntos clave y homografía para alinear y transformar las imágenes, creando una imagen panorámica final.

 El flujo del proceso incluye:
 1. Leer y preparar las imágenes.
 2. Detectar y describir los puntos clave en ambas imágenes.
 3. Coincidir los puntos clave entre las dos imágenes.
 4. Calcular la homografía para alinear las imágenes.
 5. Transformar y unir las imágenes.
 6. Recortar la imagen resultante.

In [ ]:
# %%

import cv2
import numpy as np
import matplotlib.pyplot as plt
import imageio
import imutils
cv2.ocl.setUseOpenCL(False)  # Desactiva el uso de OpenCL para evitar problemas de compatibilidad

ModuleNotFoundError: No module named 'cv2'

# %% [markdown]

 ## Selección de método de detección y coincidencia de características

 Elegimos un extractor de características (por ejemplo, 'orb') y un método de coincidencia de características (por ejemplo, 'bf' para Brute-Force).

In [ ]:
# %%

# Selección del método de extracción de características
feature_extractor = 'orb'  # Opciones: 'sift', 'surf', 'brisk', 'orb'
feature_matching = 'bf'    # Opciones de coincidencia: 'bf' para Brute-Force o 'knn' para KNN

: 

# %% [markdown]

 ## Leer y convertir las imágenes a escala de grises

 Leemos las imágenes (la imagen de consulta y la imagen de entrenamiento) y las convertimos a escala de grises para simplificar el procesamiento.

In [ ]:
# %%

# Cargar y convertir las imágenes a escala de grises
trainImg = imageio.imread('/content/drive/MyDrive/panorama(train).jpg')
trainImg_gray = cv2.cvtColor(trainImg, cv2.COLOR_RGB2GRAY)

queryImg = imageio.imread('/content/drive/MyDrive/panorama(query).jpg')
queryImg_gray = cv2.cvtColor(queryImg, cv2.COLOR_RGB2GRAY)

# Visualización de las imágenes cargadas
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, constrained_layout=False, figsize=(10,9))
ax1.imshow(queryImg, cmap="gray")
ax1.set_xlabel("Imagen de consulta (Query)", fontsize=14)
ax2.imshow(trainImg, cmap="gray")
ax2.set_xlabel("Imagen de entrenamiento (Train)", fontsize=14)
plt.show()

: 

# %% [markdown]

 ## Detección y descripción de características

 La función `detectAndDescribe` detecta puntos clave y calcula descriptores en una imagen usando el método especificado. Dependiendo de la selección (`sift`, `surf`, `brisk`, `orb`), se usa un detector/descripción diferente.

In [ ]:
# %%

def detectAndDescribe(image, method=None):
    """
    Detectar y describir puntos clave y descriptores de una imagen usando el método especificado
    """
    assert method is not None, "Debes definir un método de detección de características: 'sift', 'surf', 'brisk', 'orb'"
    
    # Selección del método de detección
    if method == 'sift':
        descriptor = cv2.xfeatures2d.SIFT_create()
    elif method == 'surf':
        descriptor = cv2.xfeatures2d.SURF_create()
    elif method == 'brisk':
        descriptor = cv2.BRISK_create()
    elif method == 'orb':
        descriptor = cv2.ORB_create()
        
    # Detección de puntos clave y cálculo de descriptores
    (kps, features) = descriptor.detectAndCompute(image, None)
    
    return (kps, features)

# Detección de puntos clave y descriptores en ambas imágenes
kpsA, featuresA = detectAndDescribe(trainImg_gray, method=feature_extractor)
kpsB, featuresB = detectAndDescribe(queryImg_gray, method=feature_extractor)

: 

# %% [markdown]

 ## Visualización de los puntos clave detectados

 Mostramos los puntos clave detectados en ambas imágenes para entender la distribución de características detectadas.

In [ ]:
# %%

# Visualizar puntos clave en ambas imágenes
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20,8), constrained_layout=False)
ax1.imshow(cv2.drawKeypoints(trainImg_gray, kpsA, None, color=(0, 255, 0)))
ax1.set_xlabel("(a) Imagen de entrenamiento", fontsize=14)
ax2.imshow(cv2.drawKeypoints(queryImg_gray, kpsB, None, color=(0, 255, 0)))
ax2.set_xlabel("(b) Imagen de consulta", fontsize=14)
plt.show()

: 

# %% [markdown]

 ## Creación del Emparejador de Características (Matcher)

 La función `createMatcher` crea y devuelve un objeto de emparejamiento de características en función del método de detección elegido.

In [ ]:
# %%

def createMatcher(method, crossCheck):
    """
    Crear y devolver un objeto Matcher
    """
    if method in ['sift', 'surf']:
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=crossCheck)
    elif method in ['orb', 'brisk']:
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=crossCheck)
    return bf

: 

# %% [markdown]

 ## Coincidencia de Puntos Clave con Brute-Force

 Usamos el emparejador Brute-Force para encontrar los puntos clave más cercanos entre las dos imágenes. La coincidencia por Brute-Force compara cada descriptor en la primera imagen con todos los descriptores en la segunda, seleccionando la coincidencia más cercana.

In [ ]:
# %%

def matchKeyPointsBF(featuresA, featuresB, method):
    bf = createMatcher(method, crossCheck=True)
        
    # Coincidir descriptores
    best_matches = bf.match(featuresA, featuresB)
    
    # Ordenar las coincidencias en orden de menor a mayor distancia
    rawMatches = sorted(best_matches, key=lambda x: x.distance)
    print("Número de coincidencias (Brute Force):", len(rawMatches))
    return rawMatches

: 

# %% [markdown]

 ## Coincidencia de Puntos Clave con KNN y Prueba de Razón

 La función `matchKeyPointsKNN` utiliza el emparejador K-Nearest Neighbor (KNN) con la prueba de razón de Lowe para mejorar la precisión y reducir coincidencias falsas.

In [ ]:
# %%

def matchKeyPointsKNN(featuresA, featuresB, ratio, method):
    bf = createMatcher(method, crossCheck=False)
    rawMatches = bf.knnMatch(featuresA, featuresB, 2)
    print("Número de coincidencias (KNN):", len(rawMatches))
    matches = []

    # Aplicar la prueba de razón
    for m, n in rawMatches:
        if m.distance < n.distance * ratio:
            matches.append(m)
    return matches

: 

# %% [markdown]

 ## Selección y Visualización de Coincidencias

 Elegimos el tipo de coincidencia (Brute-Force o KNN) y mostramos las primeras coincidencias entre las dos imágenes.

In [ ]:
# %%

print(f"Usando: {feature_matching} para el emparejamiento de características")

fig = plt.figure(figsize=(20,8))
if feature_matching == 'bf':
    matches = matchKeyPointsBF(featuresA, featuresB, method=feature_extractor)
    img3 = cv2.drawMatches(trainImg, kpsA, queryImg, kpsB, matches[:100],
                           None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
elif feature_matching == 'knn':
    matches = matchKeyPointsKNN(featuresA, featuresB, ratio=0.75, method=feature_extractor)
    img3 = cv2.drawMatches(trainImg, kpsA, queryImg, kpsB, np.random.choice(matches, 100),
                           None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plt.imshow(img3)
plt.show()

: 

# %% [markdown]

 ## Cálculo de la Homografía

 Usamos la función `getHomography` para calcular la matriz de homografía que transforma los puntos de la imagen de entrenamiento para alinearse con la imagen de consulta. Esta matriz es crucial para la transformación y unión de las imágenes.

In [ ]:
# %%

def getHomography(kpsA, kpsB, featuresA, featuresB, matches, reprojThresh):
    kpsA = np.float32([kp.pt for kp in kpsA])
    kpsB = np.float32([kp.pt for kp in kpsB])
    
    if len(matches) > 4:
        # Crear conjuntos de puntos
        ptsA = np.float32([kpsA[m.queryIdx] for m in matches])
        ptsB = np.float32([kpsB[m.trainIdx] for m in matches])
        
        # Calcular la homografía usando RANSAC
        (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)
        return (matches, H, status)
    else:
        return None

# Cálculo de la homografía
M = getHomography(kpsA, kpsB, featuresA, featuresB, matches, reprojThresh=4)
if M is None:
    print("Error en la homografía")
(matches, H, status) = M
print("Matriz de Homografía:\n", H)

: 

# %% [markdown]

 ## Aplicación de la Transformación y Unión de Imágenes

 Usamos la matriz de homografía para transformar la imagen de entrenamiento y alinearla con la imagen de consulta, creando el panorama.

In [ ]:
# %%

# Aplicar la transformación perspectiva
width = trainImg.shape[1] + queryImg.shape[1]
height = trainImg.shape[0] + queryImg.shape[0]

result = cv2.warpPerspective(trainImg, H, (width, height))
result[0:queryImg.shape[0], 0:queryImg.shape[1]] = queryImg

# Mostrar el mosaico resultante
plt.figure(figsize=(20,10))
plt.imshow(result)
plt.axis('off')
plt.show()

: 

# %% [markdown]

 ## Recorte de la Imagen Final

 Convertimos la imagen resultante a escala de grises, encontramos los contornos y recortamos la imagen para eliminar áreas en blanco.

In [ ]:
# %%

# Transformar el mosaico a escala de grises y aplicar umbral
gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]

# Encontrar contornos
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

# Obtener el contorno máximo
c = max(cnts, key=cv2.contourArea)

# Crear un rectángulo delimitador y recortar la imagen
(x, y, w, h) = cv2.boundingRect(c)
result = result[y:y + h, x:x + w]

# Mostrar la imagen final recortada
plt.figure(figsize=(20,10))
plt.imshow(result)
plt.axis('off')
plt.show()

: 

# %% [markdown]

 # Ejemplo de Mosaico de Imágenes

 Este código utiliza métodos de detección de características, coincidencia de puntos clave, homografía y transformación de perspectiva para unir varias imágenes en un mosaico.

 El flujo principal del proceso es el siguiente:
 1. **Detección de puntos clave y coincidencia**: Usamos SIFT para detectar y coincidir puntos clave entre las imágenes.
 2. **Cálculo de la homografía**: A partir de los puntos coincidentes, calculamos la matriz de homografía para alinear las imágenes.
 3. **Transformación y combinación de imágenes**: Transformamos una imagen para alinearla con otra y luego las unimos en un mosaico.
 4. **Recorte de la imagen resultante**: Eliminamos los bordes negros para obtener un mosaico limpio.

 Empezamos cargando las librerías necesarias.

In [ ]:
# %%

import numpy as np
import cv2
import matplotlib.pyplot as plt

: 

# %% [markdown]

 ## Configuración de la visualización en Jupyter

 Configuramos `matplotlib` para mostrar gráficos en línea en el notebook.

In [ ]:
# %%

%matplotlib notebook

: 

# %% [markdown]

 ## Función `getHomo`: Cálculo de Homografía

 Esta función calcula la **matriz de homografía** entre dos conjuntos de puntos, `X` y `x`, mediante un enfoque similar a la **transformación directa lineal** (DLT). La homografía permite alinear una imagen con respecto a otra basándose en los puntos coincidentes.

 ### Parámetros
 - `X`: Coordenadas de los puntos en la imagen de referencia.
 - `x`: Coordenadas de los puntos en la imagen a transformar.
 - `iterations`: Número de iteraciones para la selección aleatoria de puntos (por defecto 500).
 - `thresh`: Umbral para determinar qué puntos cumplen con la homografía (por defecto 5).

 ### Flujo
 1. Selecciona aleatoriamente 4 puntos para calcular una homografía temporal.
 2. Calcula la matriz de homografía usando la descomposición en valores singulares (SVD).
 3. Aplica la homografía calculada a todos los puntos para verificar cuántos cumplen con el umbral.
 4. Guarda la mejor matriz de homografía que maximiza el número de puntos coincidentes.

In [ ]:
# %%

def getHomo(X, x, iterations=500, thresh=5):
    maxCount = 0  # Inicializar contador máximo de coincidencias
    for i in range(iterations):
        # Selección aleatoria de 4 puntos
        idx = np.random.randint(X.shape[0], size=4)
        temp_X = X[idx]
        temp_x = x[idx]

        # Construcción de la matriz para el cálculo de homografía
        ax = np.concatenate((-temp_X, np.zeros((4, 3)), temp_x[:, 0:1] * temp_X), axis=1)
        ay = np.concatenate((np.zeros((4, 3)), -temp_X, temp_x[:, 1:] * temp_X), axis=1)
        M = np.concatenate((ax, ay), axis=0)
        
        # Descomposición en valores singulares (SVD) para calcular la homografía
        u, s, v = np.linalg.svd(M)
        H = v[8].reshape(3, 3)
        
        # Comprobación de cuántos puntos cumplen la homografía
        res = X.dot(H.T)
        res = np.divide(res, res[:, 2].reshape(-1, 1))  # Normalización
        err = np.linalg.norm(res[:, :2] - x, axis=1)
        tempCount = np.count_nonzero(err < thresh)

        # Guardar la mejor homografía
        if tempCount > maxCount:
            bestH = H
            maxCount = tempCount
    
    return bestH

: 

# %% [markdown]

 ## Función `keypts`: Detección de Puntos Clave y Coincidencias

 La función `keypts` detecta los puntos clave en dos imágenes utilizando **SIFT** y encuentra coincidencias sólidas entre ambas mediante el método de **prueba de razón**.

 ### Parámetros
 - `im1`: Primera imagen en escala de grises.
 - `im2`: Segunda imagen en escala de grises.
 - `ratio`: Ratio de prueba para filtrar coincidencias (por defecto 0.4).

 ### Salida
 - Puntos clave de `im1`.
 - Puntos clave de `im2`.
 - Lista de coincidencias válidas entre las imágenes.

In [ ]:
# %%

def keypts(im1, im2, ratio=0.4):
    # Detección de puntos clave con SIFT
    sift = cv2.xfeatures2d.SIFT_create(200)
    kp1, desc1 = sift.detectAndCompute(im1, None)
    kp2, desc2 = sift.detectAndCompute(im2, None)
    
    # Coincidencia con BFMatcher y prueba de razón
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc1, desc2, k=2)
    
    goodMatches = []
    for m, n in matches:
        if (m.distance / n.distance) < ratio:
            goodMatches.append(m)
    
    return kp1, kp2, goodMatches

: 

# %% [markdown]

 ## Función `stitch`: Unión de Imágenes

 La función `stitch` une dos imágenes. Primero detecta puntos clave y coincidencias, luego calcula la homografía para transformar una imagen con respecto a la otra.

 ### Parámetros
 - `im1`: Imagen de referencia (base).
 - `im2`: Imagen a transformar.

 ### Salida
 - Imagen resultante de la combinación de `im1` e `im2`.

In [ ]:
# %%

def stitch(im1, im2):
    # Expandir im1 para permitir la unión
    im1 = np.pad(im1, ((im1.shape[0], im1.shape[0]), (im1.shape[1], im1.shape[1]), (0, 0)), 'constant')
    
    # Detectar y coincidir puntos clave entre ambas imágenes
    kp1, kp2, goodMatches = keypts(im1, im2)
    
    # Extraer puntos de coincidencia para calcular la homografía
    src_pts = np.array([[kp2[m.trainIdx].pt[0], kp2[m.trainIdx].pt[1]] for m in goodMatches])
    dst_pts = np.array([[kp1[m.queryIdx].pt[0], kp1[m.queryIdx].pt[1]] for m in goodMatches])
    src_pts = np.concatenate((src_pts, np.ones((src_pts.shape[0], 1))), axis=1)
    
    # Calcular la homografía y aplicar la transformación perspectiva
    h = getHomo(src_pts, dst_pts)
    warped = cv2.warpPerspective(im2, h, (im1.shape[1], im1.shape[0]))
    
    # Combinar las imágenes transformadas
    warped[im1 != 0] = 0
    final = cleanse(warped + im1)
    
    return final

: 

# %% [markdown]

 ## Función `cleanse`: Recorte de Bordes

 La función `cleanse` recorta los bordes de la imagen resultante para eliminar áreas negras, mejorando la apariencia del mosaico.

In [ ]:
# %%

def cleanse(im):
    for i in range(im.shape[1] - 1, 0, -1):
        if (im[:, i] != 0).any():
            im = im[:, :i]
            break
    for i in range(0, im.shape[1]):
        if (im[:, i] != 0).any():
            im = im[:, i:]
            break
    for i in range(0, im.shape[0]):
        if (im[i] != 0).any():
            im = im[i:]
            break
    for i in range(im.shape[0] - 1, 0, -1):
        if (im[i] != 0).any():
            im = im[:i]
            break
    
    return im

: 

# %% [markdown]

 ## Función `findRoot`: Identificación de Imagen Base

 `findRoot` encuentra la mejor imagen de referencia para construir el mosaico. Esta imagen base tiene la mayor cantidad de coincidencias con las demás imágenes, asegurando una mayor calidad en el mosaico.

In [ ]:
# %%

def findRoot(images):
    numOfFeats = []
    for i in range(len(images)):
        temp = 0
        for j in range(len(images)):
            if i != j:
                kp1, kp2, goodMatches = keypts(images[i], images[j])
                temp += len(goodMatches)
        numOfFeats.append(temp)
    
    return np.argmax(np.array(numOfFeats))

: 

# %% [markdown]

 ## Ejecución del Mosaico

 Se cargan las imágenes y se organiza el orden de combinación. Luego, se aplica la función `stitch` para unirlas y construir el mosaico final.

In [ ]:
# %%

# Definir el orden de las imágenes a cargar
order = [2, 1, 4, 3, 6, 5]
images = [cv2.imread('images/img2_' + str(i) + '.png') for i in order]

# Identificar la imagen base para el mosaico
root = findRoot(images)
images[0], images[root] = images[root], images[0]

# Unión iterativa de imágenes en el orden seleccionado
l = len(images)
plt.subplot(l, 1, 1)
plt.imshow(cv2.cvtColor(images[0], cv2.COLOR_BGR2RGB))
for i in range(l - 1):
    maxLen = 0
    bestMatch = -1
    for j in range(1, len(images)):
        kp1, kp2, goodMatches = keypts(images[0], images[j])
        if len(goodMatches) > maxLen:
            maxLen = len(goodMatches)
            bestMatch = j
    
    images[0] = stitch(images[0], images[bestMatch])
    images.pop(bestMatch)
    
    plt.subplot(l, 1, i + 2)
    plt.imshow(cv2.cvtColor(images[0], cv2.COLOR_BGR2RGB))

: 